In [1]:
from text_to_speech import *

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
MODEL_FILE_PATH = './model/single_speaker_tts.pt'

text_to_speech = TextToSpeech(torch.load(MODEL_FILE_PATH), device)

In [4]:
mel_pred, attention = text_to_speech.tts('다람쥐와 호랑이')

text_to_speech.save_tts_result('./audio_sample/tts_sample.wav')

Saved tts result!
path: ./audio_sample/tts_sample.wav


In [5]:
# text_to_speech.plot_attention_weights()

In [6]:
import configs as cf
from preprocess import *
from train_tts import *

In [7]:
# Set Dataset
full_ds = get_single_speaker_dataset(
    cf.SPEAKER, cf.WAV_PATH, cf.SCRIPT_FILE_NAME,
    cf.SR, cf.N_MELS, cf.N_FFT, cf.HOP_LENGTH, cf.WIN_LENGTH
)

# Split train, valid
data_length = len(full_ds)
train_ds_len = int(data_length*cf.TRAINSET_RATIO)
valid_ds_len = data_length - train_ds_len
train_ds, valid_ds = random_split(full_ds, [train_ds_len, valid_ds_len])

# Set DataLoader
train_dl = get_dl_by_ds(train_ds, cf.BATCH_SIZE, cf.DL_NUM_WORKERS, shuffle=True)
valid_dl = get_dl_by_ds(valid_ds, cf.BATCH_SIZE, cf.DL_NUM_WORKERS, shuffle=False)

Loading ['여1_소설1', '여1_자기계발2', '여1_동화1', '여1_자기계발1'] ...
소설1 Done!
자기계발2 Done!
동화1 Done!
자기계발1 Done!


In [8]:
batch = iter(train_dl).next()

In [9]:
model = torch.load(MODEL_FILE_PATH).to(device)

In [10]:
tensor_dict_to_device(batch, device)
text_tokens, mel = batch['text_tokens'], batch['mel']
now_batch_len = len(mel)
mel_pred, _, _ = model(text_tokens, mel[:,:-1])
mel_pred_batch = mel_pred.cpu().detach().numpy()

In [11]:
def save_tts_result(mel_pred, save_file_path):
    inversed = librosa.feature.inverse.mel_to_audio(
        mel_pred.T, sr=cf.SR, hop_length=cf.HOP_LENGTH, win_length=cf.WIN_LENGTH)

    sf.write(save_file_path, inversed, cf.SR)
    print(f'Saved tts result!\npath: {save_file_path}')

In [12]:
mel_pred = mel_pred_batch[5]

In [13]:
save_tts_result(mel_pred, './audio_sample/tts_sample_by_dl.wav')

Saved tts result!
path: ./audio_sample/tts_sample_by_dl.wav
